In [2]:
import os
import argparse
import pandas as pd
import numpy as np
import pickle
from utils_test import *

In [3]:
from jellyfish._jellyfish import damerau_levenshtein_distance
import distance
from sklearn import metrics

In [4]:
from keras.models import load_model
from theano.ifelse import ifelse #added this

Using Theano backend.


In [5]:
name = 'helpdesk'
args = {
    'inputdir': '../input/{}/'.format(name),   
    'outputdir': './output_files/{}/'.format(name),
    'modelname': 'model_29-1.51.h5',
}

args = argparse.Namespace(**args)

In [6]:
test = pd.read_csv(args.inputdir+'test.csv')
test = transformDf(test)

In [7]:
with open(args.inputdir+'parameters.pkl', "rb") as f:
    maxlen = pickle.load(f)
    num_features = pickle.load(f)
    chartoindice = pickle.load(f)
    targetchartoindice = pickle.load(f)
    divisor = pickle.load(f)
    divisor2 = pickle.load(f)

In [8]:
with open(args.inputdir+'preprocessed_data.pkl', "rb") as f:
    X = pickle.load(f)
    y_a = pickle.load(f)
    y_t = pickle.load(f)
    X_test = pickle.load(f)
    y_a_test = pickle.load(f)
    y_t_test = pickle.load(f)

# Utils

In [9]:
model = load_model(args.outputdir+args.modelname)

/Users/hoangnguyen/miniconda3/envs/pydata2/lib/python2.7/site-packages/theano/tensor/basic.py:5130: UserWarning: flatten outdim parameter is deprecated, use ndim instead.
  "flatten outdim parameter is deprecated, use ndim instead.")


In [10]:
test_groupByCase = test.groupby(['CaseID'])

#get features all data
sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4 = getFeature(test_groupByCase)

#get output all data
next_chars, next_chars_t, next_chars_t2, next_chars_t3, next_chars_t4 = getOutput(test_groupByCase)

# Check longest case --> suffix: 2 to 7
test_len = findLongestLength(test_groupByCase)
test_len - 5

8

In [29]:
def get_remaining_trace_accuracy(true_labels, pred_probabilities, targetchartoindice):
    true_trace, pred_trace = getRemaingTrace(true_labels, pred_probabilities, targetchartoindice)
    count=0.0
    for i in range(len(true_trace)):
        if true_trace[i] == pred_trace[i]:
            count+=1
    acc = count/len(true_trace)
    print 'Accuracy of remaining trace: {}%'.format(acc*100)

In [11]:
def evalAct(true_label, pred_prob, targetchartoindice):
    #for log-loss: get probabilities
    gt_a = one_hot_encode(true_label, targetchartoindice)
    log_loss = metrics.log_loss(gt_a, pred_prob)
    print 'Log-loss: {}'.format(log_loss)
    
    #for accuracy: get labels
    gt_a_label = true_label
    pred_a_label = getLabel(pred_prob, targetchartoindice)
    acc = metrics.accuracy_score(gt_a_label, pred_a_label)
    print 'Accuracy: {}%'.format(acc*100)
    acc_3 = get_top3_accuracy(pred_prob, gt_a_label, targetchartoindice)
    print 'Top 3 accuracy: {}%'.format(acc_3*100)

In [12]:
def evalTime(true_time, pred_time, divisor):
    #get time
    gt_t = true_time
    pred_t = inverseTime(pred_time, divisor)
    
    mse = metrics.mean_squared_error(gt_t, pred_t)
    mae = metrics.mean_absolute_error(gt_t, pred_t)
    median = metrics.median_absolute_error(gt_t, pred_t)
    
    print 'Mean Squared Error: {0}s\t| {1} days'.format(mse, mse/86400)
    print 'Mean Absolute Error: {0}s\t| {1} days'.format(mae, mae/86400)
    print 'Median Absolute Error: {0}s\t| {1} days'.format(median, median/86400)

In [13]:
# DL distance
#distance.nlevenshtein(gt_a_label, pred_a_label) #0.1731066460587326

#damerau_levenshtein_distance(gt_a_label, pred_a_label) #784

#distance.jaccard(gt_a_label, pred_a_label) #0.4444444444444444

In [14]:
def getSuffix(suffix, sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4, next_chars, next_chars_t):
    sentences_2 = []
    sentences_t_2 = []
    sentences_t2_2 = []
    sentences_t3_2 = []
    sentences_t4_2 = []
    next_chars_2 = []
    next_chars_t_2 = []

    for i in range(len(sentences)):
        if len(sentences[i]) == suffix:
            sentences_2.append(sentences[i])
            sentences_t_2.append(sentences_t[i])
            sentences_t2_2.append(sentences_t2[i])
            sentences_t3_2.append(sentences_t3[i])
            sentences_t4_2.append(sentences_t4[i])
            next_chars_2.append(next_chars[i])
            next_chars_t_2.append(next_chars_t[i])
    return sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, next_chars_2, next_chars_t_2

# All data

## Predict

In [15]:
pred = model.predict(X_test, verbose=0)
pred_a = pred[0]
pred_t = pred[1]

In [16]:
pred[0].shape, pred[1].shape

((4529, 10), (4529, 1))

## Evaluate

### Activity

In [30]:
get_remaining_trace_accuracy(next_chars, pred_a, targetchartoindice)

Accuracy of remaining trace: 59.9369085174%


In [31]:
evalAct(next_chars, pred_a, targetchartoindice)

Log-loss: 0.556086570241
Accuracy: 82.2918966659%
Top 3 accuracy: 99.1388827556%


### Time

In [32]:
evalTime(next_chars_t, pred_t, divisor)

Mean Squared Error: 2.34726015319e+11s	| 2716736.28841 days
Mean Absolute Error: 195370.844942s	| 2.26123663128 days
Median Absolute Error: 18933.6445312s	| 0.219139404297 days


# Suffix

## Predict

In [33]:
sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, next_chars_2, next_chars_t_2 =  getSuffix(6, sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4, next_chars, next_chars_t)

In [34]:
len(sentences_2), len(next_chars_2)

(89, 89)

In [35]:
X_test = vectorizeInput(sentences_2, sentences_t_2, sentences_t2_2, sentences_t3_2, sentences_t4_2, 
                        maxlen, num_features, chartoindice, 
                        divisor, divisor2, divisor3=86400, divisor4=7)

In [36]:
pred = model.predict(X_test, verbose=0)
pred_a = pred[0]
pred_t = pred[1]

In [37]:
pred[0].shape, pred[1].shape

((89, 10), (89, 1))

## Evaluate

### Activity

In [38]:
evalAct(next_chars_2, pred[0], targetchartoindice)

Log-loss: 0.349807586835
Accuracy: 88.7640449438%
Top 3 accuracy: 100.0%


### Time

In [39]:
evalTime(next_chars_t_2, pred_t, divisor)

Mean Squared Error: 1.20616652819e+11s	| 1396026.0743 days
Mean Absolute Error: 85132.4118725s	| 0.985328841117 days
Median Absolute Error: 596.995910645s	| 0.00690967489172 days
